/tmp/ipykernel_48549/1436281817.py:154: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



     desig  eid  year  waiting
0      ACC   24  2023      0.0
1  ACC{HQ}    7  2023      0.0
2     APFC  243  2023      0.0
3   RPFC-I  163  2023      0.0
4  RPFC-II  286  2023      0.0
       desig  eid  year  waiting
147  RPFC-II  302  2052     39.0
148      ACC   32  2053      0.0
149     APFC  445  2053    177.0
150   RPFC-I  240  2053      0.0
151  RPFC-II  302  2053     34.0


       desig  eid  year waiting
109  RPFC-II  164  2051       0
110     APFC  445  2052       0
111  RPFC-II  128  2052       0
112     APFC  445  2053       0
113  RPFC-II   94  2053       0


In [ ]:
def guess_employee_by_age_and_rank(age, rank_at_apfc):
    # Find all employees at the APFC level
    apfc_employees = [emp for emp in employees if emp.desig == "APFC"]
    
    # Filter employees by age and rank
    possible_employees = [emp for emp in apfc_employees if emp.age == age]
    
    # Rank them based on their experience and check if the rank matches
    sorted_employees = sorted(possible_employees, key=lambda x: x.exp, reverse=True)
    if len(sorted_employees):
        matched_employee = sorted_employees[0]
        return matched_employee
    else:
        return f"No employee found at rank {rank_at_apfc} with age {age}."

# Example usage:
age = 26  # Example age
rank_at_apfc = 5  # Example rank (e.g., 5th in the list of employees with this age at APFC level)

employee = guess_employee_by_age_and_rank(age, rank_at_apfc)



Employee found: ID 979, Age 26, Designation APFC


Employee found: ID 979, Age 26, Designation APFC


KeyError: -1

In [33]:
import random
import pandas as pd
import plotly.express as px

# Hierarchy Information
hierarchy = {
    "APFC": {"years_exp": 5, "positions": 445, "in_position": 243, "min_age": 20, "next_level": "RPFC-II"},
    "RPFC-II": {"years_exp": 5, "positions": 302, "in_position": 286, "min_age": 27, "next_level": "RPFC-I"},
    "RPFC-I": {"years_exp": 5, "positions": 240, "in_position": 163, "min_age": 33, "next_level": "ACC"},
    "ACC": {"years_exp": 5, "positions": 32, "in_position": 24, "min_age": 47, "next_level": "ACC{HQ}"},
    "ACC{HQ}": {"years_exp": 500, "positions": 11, "in_position": 7, "min_age": 52, "next_level": None},  # No further promotion
}

# Global Variables
year = 2023
employees = []
eid_counter = 0
dfy = pd.DataFrame(columns=['desig', 'eid', 'year', 'waiting'])

class Employee:
    def __init__(self, eid, desig, exp, age):
        self.eid = eid
        self.desig = desig
        self.exp = exp
        self.age = age
        self.active = True
        self.waiting_for_promotion = 0
        self.retirement_age = 60

        # Track joining and retirement years
        self.joining_age = age - exp
        self.joining_year = year - exp  # Joining year based on experience
        self.retirement_year = self.joining_year + (self.retirement_age - self.joining_age)
        self.retirement_designation = desig  # Updated on promotion

        # Track joining ages for designations
        self.joining_as_APFC = self.joining_age if desig == "APFC" else 0
        self.joining_as_RPFCII = 0
        self.joining_as_RPFCI = 0
        self.joining_as_ACC = 0
        self.joining_as_ACCHQ = 0

    def promote(self, new_desig):
        self.desig = new_desig
        self.exp = 0  # Reset experience on promotion
        self.waiting_for_promotion = 0
        self.retirement_designation = new_desig  # Update retirement designation

        # Update joining age for new designation
        if new_desig == "RPFC-II":
            self.joining_as_RPFCII = self.age
        elif new_desig == "RPFC-I":
            self.joining_as_RPFCI = self.age
        elif new_desig == "ACC":
            self.joining_as_ACC = self.age
        elif new_desig == "ACC{HQ}":
            self.joining_as_ACCHQ = self.age

    def increment_age(self):
        self.age += 1
        if self.age > self.retirement_age:
            self.active = False

    def can_be_promoted(self, hierarchy):
        if not self.active or self.desig == "ACC{HQ}":  # Hardcode highest level
            return False
        return self.exp >= hierarchy[self.desig]["years_exp"]

    def update_promotion_timeline(self, hierarchy):
        if self.can_be_promoted(hierarchy):
            years_needed = hierarchy[self.desig]["years_exp"] - self.exp
            self.years_to_promotion = years_needed if years_needed > 0 else 0

    def __repr__(self):
        return (f"Employee(eid={self.eid}, desig='{self.desig}', exp={self.exp}, age={self.age}, "
                f"active={self.active}, joining_year={self.joining_year}, retirement_year={self.retirement_year}, "
                f"retirement_designation={self.retirement_designation}, joining_as_APFC={self.joining_as_APFC}, "
                f"joining_as_RPFCII={self.joining_as_RPFCII}, joining_as_RPFCI={self.joining_as_RPFCI}, "
                f"joining_as_ACC={self.joining_as_ACC}, joining_as_ACCHQ={self.joining_as_ACCHQ})")



# Function to Create Employees
def create_employee(desig, exp, age):
    global eid_counter
    employee = Employee(eid_counter, desig, exp, age)
    employees.append(employee)
    eid_counter += 1
    return employee

# Populate Initial Employees
# Populate Initial Employees
def populate():
    for desig, data in hierarchy.items():
        for _ in range(data["in_position"]):
            # Ensure initial experience is capped
            max_initial_exp = min(data["years_exp"], 8)
            exp = random.randint(0, max_initial_exp)
            
            # Calculate maximum possible joining age based on retirement age (60) and experience
            retirement_age = 60
            max_possible_joining_age = retirement_age - exp
            
            # Ensure joining age is within reasonable bounds (min_age and max_possible_joining_age)
            min_possible_joining_age = data["min_age"]
            joining_age = random.randint(min_possible_joining_age, max_possible_joining_age)
            
            # Calculate current age by adding experience to joining age
            current_age = joining_age + exp
            
            create_employee(desig, exp, current_age)


# Promote Eligible Employees
def promote_eligible_employees():
    for desig, data in hierarchy.items():
        next_level = data["next_level"]
        if next_level:
            vacancy = get_vacancy(next_level)
            if vacancy > 0:
                eligible_employees = sorted([emp for emp in employees if emp.desig == desig and emp.active and emp.exp >= hierarchy[desig]["years_exp"]], key=lambda x: x.exp, reverse=True)[:vacancy]
                for emp in eligible_employees:
                    emp.promote(next_level)

# Get Vacancy at a Level
def get_vacancy(level):
    total_positions = hierarchy[level]["positions"]
    filled_positions = sum(1 for emp in employees if emp.desig == level and emp.active)
    return total_positions - filled_positions

# Year Change Function
def year_change(num_years):
    global year, dfy
    for _ in range(num_years):
        for emp in employees:
            emp.increment_age()
            if emp.active and emp.exp > 7 and emp.desig in ['APFC','RPFC-II','RPFC-I','ACC']:
                emp.waiting_for_promotion = 1
        promote_eligible_employees()
        year += 1

        # Tracking yearly data
        df_year = pd.DataFrame([{'desig': emp.desig, 'eid': emp.eid, 'year': year, 'waiting': emp.waiting_for_promotion} for emp in employees if emp.active])
        pivot_table = pd.pivot_table(df_year, values=['eid','waiting'], index=['desig'], aggfunc={'eid':'count','waiting':'sum'}, margins=False).reset_index()
        pivot_table["year"] = year
        dfyt = pivot_table[['desig','eid','year','waiting']]
        dfy = pd.concat([dfy, dfyt], ignore_index = True)

# Get Seniority List at Any Level
def get_seniority_list(level):
    # Get employees at this level, sorted by experience (seniority)
    level_employees = sorted([emp for emp in employees if emp.desig == level and emp.active], key=lambda x: x.exp, reverse=True)
    return [(emp.eid, emp.desig, emp.exp, emp.age) for emp in level_employees]

# Simulate Career for an Employee
def simulate_career(employee):
    promotions = {}
    current_age = employee.age
    current_desig = employee.desig
    current_exp = employee.exp
    
    # Simulate promotions until retirement
    while employee.active:
        # Check if the employee is eligible for promotion at this level
        if current_desig in hierarchy and hierarchy[current_desig]["next_level"]:
            if current_exp >= hierarchy[current_desig]["years_exp"]:
                next_level = hierarchy[current_desig]["next_level"]
                
                # Calculate the age at which the promotion will happen
                years_to_promotion = hierarchy[current_desig]["years_exp"] - current_exp
                promotion_age = current_age + years_to_promotion
                promotions[current_desig] = promotion_age
                
                current_age = promotion_age
                current_desig = next_level
                current_exp = 0  # Reset experience after promotion
            else:
                break
        else:
            break
    
    # After all promotions, check the designation at retirement age (60)
    if current_age >= 60:
        return promotions, current_desig
    else:
        return promotions, current_desig

# Example of Querying and Career Simulation
def guess_employee_by_id(eid):
    employee = next((emp for emp in employees if emp.eid == eid), None)
    if employee:
        promotions, retirement_designation = simulate_career(employee)
        return promotions, retirement_designation
    else:
        return "Employee not found."



populate()
year_change(30)

# Check details of first 5 employees
for emp in employees:
    print(emp)

Employee(eid=0, desig='APFC', exp=3, age=75, active=False, joining_year=2020, retirement_year=2038, retirement_designation=APFC, joining_as_APFC=42, joining_as_RPFCII=0, joining_as_RPFCI=0, joining_as_ACC=0, joining_as_ACCHQ=0)
Employee(eid=1, desig='APFC', exp=1, age=81, active=False, joining_year=2022, retirement_year=2032, retirement_designation=APFC, joining_as_APFC=50, joining_as_RPFCII=0, joining_as_RPFCI=0, joining_as_ACC=0, joining_as_ACCHQ=0)
Employee(eid=2, desig='APFC', exp=0, age=71, active=False, joining_year=2023, retirement_year=2042, retirement_designation=APFC, joining_as_APFC=41, joining_as_RPFCII=0, joining_as_RPFCI=0, joining_as_ACC=0, joining_as_ACCHQ=0)
Employee(eid=3, desig='APFC', exp=4, age=90, active=False, joining_year=2019, retirement_year=2023, retirement_designation=APFC, joining_as_APFC=56, joining_as_RPFCII=0, joining_as_RPFCI=0, joining_as_ACC=0, joining_as_ACCHQ=0)
Employee(eid=4, desig='APFC', exp=0, age=76, active=False, joining_year=2023, retirement


Employee 0 Details:
Initial Designation: APFC
Joined at Age: 83
Initial Experience: 2

Promotions Timeline:

Designation at retirement (Age 60): APFC

Employee 1 Details:
Initial Designation: APFC
Joined at Age: 83
Initial Experience: 1

Promotions Timeline:

Designation at retirement (Age 60): APFC

Employee 2 Details:
Initial Designation: APFC
Joined at Age: 82
Initial Experience: 2

Promotions Timeline:

Designation at retirement (Age 60): APFC

Employee 3 Details:
Initial Designation: APFC
Joined at Age: 87
Initial Experience: 4

Promotions Timeline:

Designation at retirement (Age 60): APFC

Employee 4 Details:
Initial Designation: APFC
Joined at Age: 80
Initial Experience: 3

Promotions Timeline:

Designation at retirement (Age 60): APFC

Employee 5 Details:
Initial Designation: APFC
Joined at Age: 80
Initial Experience: 4

Promotions Timeline:

Designation at retirement (Age 60): APFC

Employee 6 Details:
Initial Designation: APFC
Joined at Age: 82
Initial Experience: 0

Promot

In [ ]:
# from google.colab import drive
# #drive.mount('drive')
# dfr = df[df['active']==False]
# dfr.to_csv('/content/drive/My Drive/retired.csv', encoding='utf-8', index=False)